# Generate Lorentz Attractor point data

In [4]:
%load_ext Cython

In [8]:
%%cython
#!python
#cython: boundscheck=False, initializedcheck=False, cdivision=True, nonecheck=False, linetrace=False, wraparound=False






# Preferences:

FILE_NAME = 'L_data.npy' ## file name to save data as a string: "example.npy"

# constants:

SIGMA = 10
RHO = 28
B = 8/3

# initial point:

X = 1
Y = 1
Z = 1

# time range:

t0 = 0
t1 = 50

# number of points to generate:

N = 5000








import numpy as np
cimport numpy as np

cdef np.ndarray f(np.ndarray v):
    """
    return a 3-dim numpy array of the 3 attractor differential 
    equations evaluated at the given (x,y,z) point
    """
    
    # constants
    cdef np.float sigma = float(SIGMA)
    cdef np.float r = float(RHO)
    cdef np.float b = float(B)
    
    return np.array([sigma * (v[1] - v[0]),
                     v[0] * (r - v[2]) - v[1] ,
                     v[0]*v[1] - b*v[2]], dtype=np.float)



cdef np.ndarray get_points(np.float t0, np.float t1, int N):
    """
    generate an `N` by 3 numpy array whose rows are approximated 
    points (x,y,z) on the attractor
    """
    cdef int i
    cdef np.float h
    cdef np.ndarray points
    cdef np.ndarray v, k1, k2, k3, k4
    
    h = (t1 - t0) / N
    
    # initialize vector array
    points = np.zeros((N, 3), dtype=np.float)

    # runge-kutta
    points[0] = np.array([float(X), float(Y), float(Z)], np.float)
    for i in range(1, N):
        v = points[i - 1] 
        k1 = h * f(v)
        k2 = h * f(v + k1 * 0.5)
        k3 = h * f(v + k2 * 0.5)
        k4 = h * f(v + k3)
        points[i] = v + (k1 + 2 * k2 + 2 * k3 + k4) / 6
    
    return points

# generate and save points
points = get_points(float(t0), float(t1), int(N))
np.save(FILE_NAME, points)
